In [8]:
import pdfplumber
import logging
import os
from typing import List

def leer_pdf(folder_name: str) -> List[str]:
    logging.info("************************LEYENDO PDFS************************")
    pdfs = []

    for archivo in os.listdir(folder_name):
        if archivo.endswith(".pdf"):
            pdfs.append(os.path.join(folder_name, archivo))

    logging.info("************************FINALIZANDO PDFS************************")
    return pdfs

pdfs = leer_pdf("newspaper")

from tika import parser
from bs4 import BeautifulSoup

# Parse with XML content
parsed = parser.from_file(pdfs[0], xmlContent=True)
xml = parsed['content']

# Use BeautifulSoup to parse XML
soup = BeautifulSoup(xml, 'lxml')

# Find all <div> elements with class 'page'
pages = soup.find_all('div', {'class': 'page'})

# Show only the second page (if it exists)
if len(pages) > 1:
    second_page = pages[0]
    text = second_page.get_text(separator='\n', strip=True)
    print(f"\n--- Page 2 ---\n{text}")
else:
    print("Second page not found.")



--- Page 2 ---
JUEVES 10 DE MARZO DE 2022 ANO CVI NO, 37,652 SAN SALVADOR, EL SALVADOR, C. A. $0.50

LA PRENSA 7

FICA

RA. DE VERDAD

EMBARAZO INFANTIL
AUMENTO EN 2021

> DATOS DEL MINSAL REPORTARON 410 EMBARAZOS EN MENORES DE 14 ANOS > 2

oo \ a ! BELFAST IESEHWAK QUE
HONRAR DEUDA >4

NACION ;

LA REINSERCION TRAS
PURGAR UNA PENA
QUE NO DEBIO SER > 6

ACUSAN A EXMINISTRA
DE PERSECUCION A
LOS SINDICALISTAS > &

a

ai ee

- a Vole : | MUJERES DENUNCIAN
Wore | =| EN MAQUILAS > 10

PIDEN ENTREVISTA DE
TESTIGO CRITERIADO
EN CASO SAQUEO > 11

ay~...UNA ACTUACION _

y ~~ INOLVIDABLE DE wee Eso que quieres
KARIMBENZEMABOTO  __\\ S38. | iEsta aqui!
AKYLIANMBAPPEY «<~ \& 4 SS
ASU PSG DE LAUEFA x oh, , Jf, Nos alegrard verte
‘CHAMPIONS L. > 46 we , Mundo ~~—S

www.laprensagrafica.com log.laprensagrafica © @prensagrafica (y) laprensagraficavideo @


In [ ]:
def limpiar_texto(texto):
    import re
    # Quitar guiones seguidos de salto de línea y unir palabras partidas
    texto = re.sub(r'(\w+)-\n(\w+)', r'\1\2', texto)
    # Unir líneas que no terminan en punto o coma, suponiendo que siguen en la misma frase
    texto = re.sub(r'(?<![.,])\n', ' ', texto)
    return texto



 2 Jueve s10de m a rzo de 2 02 2 l ap re n s ag ra f ica .com 410 MENORES DE 14 AÑOS R E P O RTA RO N EMBARA ZO EN 2021 LOS DATOS CORRESPONDEN AL PERÍODO ENTRE ENERO A SEPTIEMBRE Y SIGNIFICAN UN ALZA DEL 8 % RESPECTO A 2020.
EL MINSAL NO REPORTA NINGUNO DE ESTOS EMBARAZOS COMO PRODUCTO DE VIOLENCIA SEXUAL.
Evelyn Machuca s o c i al @laprensagrafica .com PR E SU PU E STOS Las representantes del Foro n totalde 410 niñasy adoles- tema de salud pública y no es una priori- Nacional de Salud, de la ción integral en la sexualidad y que consiU centesentre 10y14 añosins- dad para las autoridades, de forma que no Agrupación Ciudadana y de deró importante para que las niñas y adocribieron sus embarazos en el se ha puesto la mirada en la prevención de la Colectiva Feminista lescentes puedan identificar y denunciar Ministerio de Salud (MINSAL) solo entre estosembarazos. Hemosvistoque enal- señalaron reducciones en los abusos sexuales, y también prevenir los enero y septiembre de2021. La cifra re- g

In [1]:
#Downloaded libraries: langchain and langchain-community
from langchain.document_loaders import PyPDFLoader
import os

carpeta = "newspaper"

# Lista para guardar los nombres de los archivos PDF encontrados
pdfs = []

# Recorremos los archivos en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith(".pdf"):
        pdfs.append(os.path.join(carpeta, archivo))

# Mostramos los archivos encontrados
print("PDFs encontrados:")
for pdf in pdfs:
    print(pdf)


PDFs encontrados:
newspaper\periodico-1-12.pdf


In [16]:
import logging

# Configuración básica del logger
logging.basicConfig(
    level=logging.INFO,  # Puedes cambiar a DEBUG, WARNING, ERROR, etc.
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='app.log',  # Opcional: guarda en archivo
    filemode='w'         # 'w' para sobrescribir, 'a' para agregar
)

# Ejemplo de uso
logging.info("Inicio del procesamiento")
logging.warning("Esto es una advertencia")
logging.error("Esto es un error")


In [1]:
from langchain.document_loaders import PyPDFLoader
import requests

# Cargar PDF
loader = PyPDFLoader('periodico.pdf')
pages = loader.load()

# Número de páginas por bloque
bloque_tamaño = 2
total_paginas = len(pages)

# Lista para almacenar texto por bloque. 
bloques_texto = []
structured_text = []

#Llamada a LLM
def gemma(text: str) -> str:
    url = "http://localhost:11434/api/generate"
    
    prompt = f"""Reorganiza el siguiente texto para asegurar que tenga coherencia semántica y fluya de forma lógica, manteniendo el idioma original (español) y sin agregar, eliminar, resumir ni reformular el contenido; únicamente debes reordenar las partes del texto según sea necesario para mejorar su coherencia.
    Texto: {text}
"""

    payload = {
        "model": "gemma2:9b",  
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json()["response"]
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

# Procesar en bloques de n páginas
for i in range(0, total_paginas, bloque_tamaño):
    bloque = pages[i:i + bloque_tamaño]
    texto_bloque = '\n\n'.join([p.page_content for p in bloque])
    bloques_texto.append(texto_bloque)
    
    if __name__ == "__main__":
        result = gemma(bloques_texto[-1])  # Usamos el último elemento
        structured_text.append(result)
        print("Texto reorganizado:\n")
        print(result)


# bloques_texto[0] contiene las primeras 4 páginas, bloques_texto[1] las siguientes, etc.
for idx, texto in enumerate(bloques_texto):
    print(f"--- Bloque {idx+1} ---\n{texto}\n")


Texto reorganizado:

**2 de marzo de 2022 | laprensagrafica.com**

**410 MENORES DE 14 AÑOS REPORTARON EMBARAZO EN 2021**


LOS DATOS CORRESPONDEN AL PERIODO ENTRE ENERO A SEPTIEMBRE Y SIGNIFICAN UN ALZA DEL 8% RESPECTO A 2020. EL MINSAL NO REPORTO NINGUNO DE ESTOS EMBARAZOS COMO PRODUCTO DE VIOLENCIA SEXUAL.
Evelyn Machuca
social @laprensagrafica.com

Un total de 410 niñas y adolescentes entre 10 y 14 años inscribieron sus embarazos en el Ministerio de Salud (MINSAL) solo entre enero y septiembre de 2021. La cifra representa un alza del 8% respecto al mismo período del año anterior, es decir, 31 casos más que las 379 inscripciones prenatales de 2020 en ese mismo grupo de edad, según las últimas estadísticas publicadas en el portal de Transparencia de la referida cartera de Estado.

La causa principal del aumento en la cifra, y del fenómeno en sí mismo, es la ausencia de una ley de educación integral de salud sexual y reproductiva, advirtieron a LA PRENSA GRÁFICA representantes del For

In [12]:
print(structured_text[0])

**2 de marzo de 2022 | laprensagrafica.com**

**410 MENORES DE 14 AÑOS REPORTARON EMBARAZO EN 2021**


LOS DATOS CORRESPONDEN AL PERIODO ENTRE ENERO A SEPTIEMBRE Y SIGNIFICAN UN ALZA DEL 8% RESPECTO A 2020. EL MINSAL NO REPORTO NINGUNO DE ESTOS EMBARAZOS COMO PRODUCTO DE VIOLENCIA SEXUAL.
Evelyn Machuca
social @laprensagrafica.com

Un total de 410 niñas y adolescentes entre 10 y 14 años inscribieron sus embarazos en el Ministerio de Salud (MINSAL) solo entre enero y septiembre de 2021. La cifra representa un alza del 8% respecto al mismo período del año anterior, es decir, 31 casos más que las 379 inscripciones prenatales de 2020 en ese mismo grupo de edad, según las últimas estadísticas publicadas en el portal de Transparencia de la referida cartera de Estado.

La causa principal del aumento en la cifra, y del fenómeno en sí mismo, es la ausencia de una ley de educación integral de salud sexual y reproductiva, advirtieron a LA PRENSA GRÁFICA representantes del Foro Nacional de Salud (

In [14]:
import re
from datetime import datetime

texto = structured_text[0]

# Expresión regular para buscar fechas tipo "21 de diciembre de 2025"
pattern = r"(\d{1,2}) de (enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre) de (\d{4})"

match = re.search(pattern, texto, re.IGNORECASE)
if match:
    dia = int(match.group(1))
    mes_texto = match.group(2).lower()
    anio = int(match.group(3))

    # Diccionario para convertir mes en texto a número
    meses = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12
    }

    mes = meses[mes_texto]

    fecha = datetime(anio, mes, dia)
    strdate = fecha.strftime("%Y-%m-%d")
    print("Fecha extraída:", strdate)
else:
    print("No se encontró ninguna fecha.")


Fecha extraída: 2022-03-02


In [6]:
import requests

#Variables
import requests

total_pages = len(structured_text)
size = 2

json_news = []
bloques_news = []

def gemma_news(text: str) -> str:
    url = "http://localhost:11434/api/generate"
    
    prompt = f"""
Separate the text into each news article. The output should be a JSON array, where each news item contains a "Titular" and a "Content". DO NOT repeat any news articles.
Text: {text}
"""

    payload = {
        "model": "gemma2:9b",  
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json()["response"]
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

if __name__ == "__main__":
    for i in range(0, total_pages, size):
        bloque = structured_text[i:i + size]
        texto_bloque = '\n\n'.join([p for p in bloque])
        bloques_news.append(texto_bloque)

        response = gemma_news(texto_bloque)

        # Extract JSON array from response
        start_index = response.find("[")
        end_index = response.rfind("]") + 1

        json_message = response[start_index:end_index]
        json_news.append(json_message)
        print(json_message)


[
  {
    "Titular": "410 MENORES DE 14 AÑOS REPORTARON EMBARAZO EN 2021",
    "Content": "Un total de 410 niñas y adolescentes entre 10 y 14 años inscribieron sus embarazos en el Ministerio de Salud (MINSAL) solo entre enero y septiembre de 2021. La cifra representa un alza del 8% respecto al mismo período del año anterior, es decir, 31 casos más que las 379 inscripciones prenatales de 2020 en ese mismo grupo de edad, según las últimas estadísticas publicadas en el portal de Transparencia de la referida cartera de Estado.\n\nLa causa principal del aumento en la cifra, y del fenómeno en sí mismo, es la ausencia de una ley de educación integral de salud sexual y reproductiva, advirtieron a LA PRENSA GRÁFICA representantes del Foro Nacional de Salud (FNS), de la Colectiva Feminista y de la Agrupación Ciudadana para la Despenalización del Aborto.\n\nA su juicio, existen iniciativas que tienen que ver con el trabajo en niñas y adolescentes mujeres, pero no es un trabajo articulado de todas

In [15]:
import json
import re
from datetime import datetime

# Fecha simulada (puede venir de un texto con extracción previa)
fecha_str = strdate

parsed_news = []

for item in json_news:
    try:
        noticias = json.loads(item)  # Cada item es un array de noticias
        for noticia in noticias:
            if isinstance(noticia, dict):
                noticia["fecha"] = fecha_str
        parsed_news.extend(noticias)
    except json.JSONDecodeError as e:
        print("Error al decodificar este item, será omitido:\n", item)
        continue

# Guardar los datos válidos con fecha en un archivo
with open("noticias.json", "w", encoding="utf-8") as file:
    json.dump(parsed_news, file, ensure_ascii=False, indent=2)



Error al decodificar este item, será omitido:
 [
  {
    "Titular": "410 MENORES DE 14 AÑOS REPORTARON EMBARAZO EN 2021",
    "Content": "Un total de 410 niñas y adolescentes entre 10 y 14 años inscribieron sus embarazos en el Ministerio de Salud (MINSAL) solo entre enero y septiembre de 2021. La cifra representa un alza del 8% respecto al mismo período del año anterior, es decir, 31 casos más que las 379 inscripciones prenatales de 2020 en ese mismo grupo de edad, según las últimas estadísticas publicadas en el portal de Transparencia de la referida cartera de Estado.\n\nLa causa principal del aumento en la cifra, y del fenómeno en sí mismo, es la ausencia de una ley de educación integral de salud sexual y reproductiva, advirtieron a LA PRENSA GRÁFICA representantes del Foro Nacional de Salud (FNS), de la Colectiva Feminista y de la Agrupación Ciudadana para la Despenalización del Aborto.\n\nA su juicio, existen iniciativas que tienen que ver con el trabajo en niñas y adolescentes muj